El método de Euler está dado por $y_{n+1}=y_n+h\ f(x_n,y_n)$. Recordamos que para resolver la ecuación diferencia se requiere realizar la siguiente integración $\int_{x_n}^{x_{n+1}}y'(x)=\int_{x_n}^{x_{n+1}}f(x,y)dx$, donde se integra de manera recursiva la función de $f$, avanzando del intervalo $[x_n,x_{n+1}]$ al intervalo $[x_n,x_{n+1}]$. Al integrar el lado izquierdo se obtiene que $y(x_{n+1})=y_{n+1}=y_n+\int_{x_n}^{x_{n+1}}f(x,y)dx$. 

Para aumentar el orden del método se puede considerar $f(x,y)=f(x_n,y_n)+f'(x_n,y_n)h$ donde $\frac{d}{dx}f(x,y)=\frac{\partial f}{\partial x}+\frac{\partial f}{\partial y}\frac{\partial y}{\partial x} = \frac{\partial f}{\partial x}+\frac{\partial f}{\partial y}f = (\nabla{f})\cdot(1,f)$. Con esto, se puede integrar la ecuación diferencial al calcular de manera iterada $y_{n+1}=y_n+h\ f(x_n,y_n)+\frac{h^2}{2}(\frac{\partial f}{\partial x}+\frac{\partial f}{\partial y}f)\Big|_{x_n,y_n}$

Del mismo modo para considerar un orden mayor se calcula la segunda derivada mediante $\frac{d^2}{dx^2}f(x,y)=\frac{d}{dx}(\frac{\partial f}{\partial x}+\frac{\partial f}{\partial y}f)=\underbrace{ \frac{\partial^2 f}{\partial x^2}+\frac{\partial ^2 f}{\partial x \partial y}f+(\frac{\partial^2 f}{\partial x \partial y}+\frac{\partial^2 f}{\partial y^2}f)f}_\text{Matriz Hessiana} +(\underbrace{\frac{\partial f}{\partial x}+\frac{\partial f}{\partial y}f}_\text{$\nabla f \cdot (1, f)$})^2$.


Esta forma de hacer los cálculos complica el método pues se utilizan derivadas de orden superior y aparecen el Gradiente y la matriz Hessiana sólo con tomar orden dos. Por esta razón se puede utilizar en segundo orden el método de Runge Kutta.

Primero resolveremos una ecuación diferencial de la forma $x´=f(x,t)$ utilizando ambos métodos para comparar su efectividad. En particular consideraremos $x'=-tx$, con condición inicial $x_0=x(t=0)=1$, la cual tiene como solución $e^{-t^2}$

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
def ecdif(x,t):
    return -t*x

In [ ]:
def euler (funcion,x0,t0,tf,h=0.01):
    tiempos=np.arange(t0,tf,h)
    x=np.zeros(len(tiempos))
    x[0]=x0
    for i in range(0,len(tiempos)-1):
        x[i+1]=x[i]+h*funcion(x[i],tiempos[i])
    solucion=[tiempos,x]    
    return solucion

Para el segundo método calcularemos las derivadas parciales de $x'=f(x,t)$ respecto de x y de t. Se tiene que $\partial_{x}f=-t$ y $\partial_{t}f=-x$.

In [ ]:
def dert(x,t):
    return -x

def derx(x,t):
    return -t

In [ ]:
def orden2 (funcion,dert,derx,x0,t0,tf,h=0.01):
    tiempos=np.arange(t0,tf,h)
    x=np.zeros(len(tiempos))
    x[0]=x0
    for i in range(0,len(tiempos)-1):
        x[i+1]=x[i]+h*funcion(x[i],tiempos[i])+(0.5*h**2)*(dert(x[i],tiempos[i])+funcion(x[i],tiempos[i])*derx(x[i],tiempos[i]))
    solucion=[tiempos,x]    
    return solucion

In [ ]:
sol1=euler(ecdif,1,0,1,.1)
sol2=orden2(ecdif,dert,derx,1,0,1,.1)
plt.plot(sol1[0],sol1[1],label="euler")
plt.plot(sol2[0],sol2[1],label="orden 2")
plt.plot(sol2[0],np.exp(-sol1[0]**2),label="exacta")
plt.legend()

## Integración en Varias Dimensiones

Anteriormente habíamos utilizado la siguiente expresión para realizar una integración a segundo orden: $f(x,y)=f(x_n,y_n)+f'(x_n,y_n)h$ donde $\frac{d}{dx}f(x,y)=\frac{\partial f}{\partial x}+\frac{\partial f}{\partial y}\frac{\partial y}{\partial x} = \frac{\partial f}{\partial x}+\frac{\partial f}{\partial y}f = (\nabla{f})\cdot(1,f)$. Para integrar se tenía que $y_{n+1}=y_n+h\ f(x_n,y_n)+\frac{h^2}{2}(\frac{\partial f}{\partial x}+\frac{\partial f}{\partial y}f)\Big|_{x_n,y_n}$.

Para realizar una integración en muchas variables, se cambia la variable dependiente $y$ por un vector $\vec{y}$ y 


In [ ]:
def integra2(ec,d_ec,p_ini,x,h=0.1):
    #f=ec(p_ini)
    #df=d_ec(p_ini)[0]
    #jac=d_ec(p_ini)[0]
    tiempos = np.arange(x[0],x[1]+h,h)
    sol = np.zeros((len(tiempos),len(p_ini)))
    sol[0,:] = p_ini
    for i in xrange(len(tiempos)-1):
        f=ec(sol[i,:],tiempos[i])
        df,jac=d_ec(sol[i,:],tiempos[i])
        sol[i+1,:]=sol[i,:]+h*f+(0.5*h*h*(jac[0] + np.dot(jac[1],f)))
    return tiempos,sol


def prueba(y,t):
    return np.array(y)

def d_prueba(y,t):
    return np.array([0,0]),np.array([[0,1],[-1,0]])

t,y=integra2(prueba,d_prueba,[1,0],[0,100],0.1)
#plt.plot(t,y[:,0],t,np.cos(t))
plt.plot(y[:,0],y[:,1])
plt.show()


## Oscilador Harmónico Forzado 

El oscilador harmónico sin forzamiento está se encuentra dado por:

In [ ]:
def prueba(y,t):
    return np.array([y[1],-y[0]])
def d_prueba(y,t):
    return np.array([0,0]),np.array([[0,1],[-1,0]])

t,y=integra2(prueba,d_prueba,[1,0],[0,50],0.1)
plt.plot(y[:,0],y[:,1])

Para el oscilador forzado tomaremos el siguiente ejemplo para el método:

$$
y\prime\prime+2\gamma y\prime + w_{0}^{2} sin(y) = sin(t)
$$

Simplificado por cambio de variable:

$$
u\prime = -2\gamma u - w_{0}^{2} sin(y) + sin(t)
$$

De manera que la parcial y el Jacobiano son:
$$
\frac{\partial f }{\partial t} = (0,cos(t)) \\
J = \begin{pmatrix} 0 & 1 \\ -0.5cos(t) & -2 \end{pmatrix}
$$

In [ ]:
def p(y,t):
    return np.array([y[1],-2*y[1]-0.5*np.sin(y[0])+np.exp(-t)])

def dp(y,t):
    return np.array([0,-np.exp(-t)]),np.array([[0,1],[-0.5*np.cos(y[0]),-2]])

t,y=integra2(p,dp,[1,0],[0,50],0.01)
plt.plot(t,y[:,0])

In [ ]:
def p(y,t):
    return np.array([y[1],-2*y[1]-0.5*np.sin(y[0])+np.sin(-t)])

def dp(y,t):
    return np.array([0,np.cos(t)]),np.array([[0,1],[-0.5*np.cos(y[0]),-2]])

t,y=integra2(p,dp,[1,0],[0,50],0.01)
plt.plot(t,y[:,0])

In [ ]:
plt.plot(y[:,0],y[:,1])

# Método de Runge-Kutta de 2o Orden

La solución exacta de una ecuación diferencial está dada por $\vec{y_{n+1}}= \vec{y_n}+\int_{x_n}^{x_{n+1}}\vec{f}(x,\vec{y})dy$. La expansión de Taylor alrededor de $x_0$ está dada por: $\vec{f}=\vec{f}(x_0)+\vec{f}'(x_0) x + \mathcal{O}(h^2)$. La derivada de la función la podemos evaluar en el punto medio del intervalo a integrar $[x_n,x_{n+1}]$. Con esto se tiene que  

Entonces, se tiene que $\vec{y}_{n+1}=\vec{y_n}+h\ \vec{f}(x_{n+\frac{1}{2}},\vec{y}_{n+\frac{1}{2}})$, donde $x_{n+\frac{1}{2}}=x_n+\frac{h}{2}$. Entonces, por Euler, podemos aproximar el valor de $\vec{y}$ en le punto medio como $\vec{y}_{n+\frac{1}{2}}=\vec{y}_n+\frac{h}{2}\vec{f}(x_n,y_n)$. Por tanto, el método de Runge-Kutta de 2o orden queda dado por:

$$\vec{y}_{n+1}=\vec{y}_n+h\vec{f}(x_n+\frac{h}{2},\vec{y_n}+\frac{h}{2}\vec{f}(x_n,y_n))$$

In [ ]:
"""
def runge_kutta(ec_dif,x0,t0,tf,h=0.1):
    tiempos=np.arange(t0,tf+h,h)
    sol=np.zeros((len(tiempos),len(x0)))
    sol[0,:]=x0
    for i in xrange(len(tiempos)-1):
        sol[i+1,:]=sol[i,:]+h*ec_dif(tiempos[i]+0.5*h,sol[i,:]+0.5*h*ec_dif(tiempos[i],sol[i,:]))
    return tiempos,sol   
"""

def runge_kutta(ec,p_ini,x,h=0.1):
    tiempos = np.arange(x[0],x[1]+h,h)
    sol = np.zeros((len(tiempos),len(p_ini)))
    sol[0,:] = p_ini
    for i in xrange(len(tiempos)-1):
        f=ec(sol[i,:],tiempos[i])
        f_next = ec(sol[i,:]+(h/2)*f,tiempos[i]+h/2)
        y_next = sol[i,:] + h*f_next
        sol[i+1,:]=y_next
    return tiempos,sol

Regresamos al ejemplo anterior.

Notemos la ventaja de que no hay que utilizar el Jacobiano ni la parcial. El método es agnóstico de las derivadas analíticas de nuestras ecuaciones diferenciales.

In [ ]:
def p(y,t):
    return np.array([y[1],-2*y[1]-0.5*np.sin(y[0])+np.sin(t)])
t,y = runge_kutta(p,[1,0],[0,100],0.1)
plt.plot(y[:,0],y[:,1])

## Comparación de Métodos de Integración

Deseamos resolver el problema gravitacional, donde la ecuación de movimiento está dada por: $\frac{d\vec{p}}{dt}=-\frac{G M m}{\left\lVert \vec{r}-\vec{r}_m \right\rVert ^2}\hat{(\vec{r}-\vec{r}_m)}$. Entonces, vemos que, si $\vec{r}_m$ y la masa es constante, la ecuación queda como $\frac{d^2\vec{r}}{dt^2}=-\frac{G M }{r^2}\hat{r}$. Ahora, hacemos los siguientes cambios de variable:

$$t_1=t \sqrt{GM} \Rightarrow \frac{d}{dt}=\frac{dt_1}{dt}\frac{d}{dt_1}=\sqrt{GM}\frac{d}{dt_1} \Rightarrow \frac{d^2}{dt}=GM \frac{d^2}{dt_1^2}$$

$$\vec{\rho}=\frac{\vec{r}}{R_T} \Rightarrow \frac{d^2\vec{\rho}}{dt}=-\frac{GM}{\rho^2 R_T^3}\hat{\rho} $$

$$\tau = \sqrt{\frac{GM}{R_T^3}}t \Rightarrow \frac{d^2}{dt^2}=\frac{GM}{R_T^3}\frac{d^2}{d \tau^2}$$

Al combinar estos cambios de variable se obtiene que:

$$\frac{d^2\vec{\rho}}{d\tau ^2}=-\frac{\hat{\rho}}{\rho^2}$$

Además, vemos que si hay un movimiento periódico que tarda $t=2\pi$ se tiene que $\tau^2=\frac{(2\pi)^2}{GM}R_T^3$ (3a Ley de Kepler)

Ahora, deseamos reolver la ecuación $\frac{d^2\vec{\rho}}{d\tau ^2}=-\frac{\hat{\rho}}{\rho^2}$ donde $\vec{\rho}=(\ddot{\rho}_x,\ddot{\rho}_y)=-\frac{\vec{\rho}}{\rho^3}$. Ahora, hacemos los siguientes cambios de variable para reducir el órden de la ecuación diferencial:

$$\dot{\rho_x}=v_{\rho x} \qquad \dot{\rho_x}=v_{\rho x} \qquad \dot{v}_{\rho x}=-\frac{\rho_x}{\rho^3} \qquad \dot{v}_{\rho y}=-\frac{\rho_y}{\rho^3}, \qquad donde \quad \rho=\sqrt{\rho_x^2+\rho_y^2}$$

### Utilizamos el Método de Segundo Orden "Euler de 2o Órden"

El Jacobiano del sistema es 
$$\vec{J}=\begin{pmatrix}0 & 0 & 1 & 0 \\ 0 & 0 & 0 & 1 \\ \frac{2\rho_x^2-\rho_y^2}{\rho^5} & \frac{3\rho_x\rho_y}{\rho^5} & 0 & 0 \\ \frac{3\rho_x\rho_y}{\rho^5} & \frac{2\rho_y^2-\rho_x^2}{\rho^5} & 0 & 0 \end{pmatrix}$$

In [ ]:
def mov_planetario(x, t):
    r3 = (x[0]*x[0]+x[1]*x[1])**(1.5)
    drx = x[2]
    dry = x[3]
    dvrx = -x[0]/r3
    dvry = -x[1]/r3
    return np.array([drx, dry, dvrx, dvry])

def J_mov_planetario(x, t):
    d_ec = np.array([0,0,0,0])
    r5 = (x[0]*x[0]+x[1]*x[1])**(2.5)
    rxmry = (2*x[0]*x[0]-x[1]*x[1])/r5
    rymrx = (2*x[1]*x[1]-x[0]*x[0])/r5
    rxry = (3*x[0]*x[1])/r5
    Jac = np.array([[0,0,1,0],[0,0,0,1],[rxmry, rxry, 0,0],[rxry,rymrx,0,0]])
    return d_ec, Jac

t, r_vr = integra2(mov_planetario, J_mov_planetario, np.array([1.0, 0.0, 0.1, 0.5]), [0,2*np.pi], 1.0e-4)

fig = plt.figure()
ax = fig.add_subplot(111)

ax.plot(r_vr[:,0], r_vr[:,1])
ax.set_aspect('equal');

### Utililizamos Runge-Kutta de 2o Orden

In [ ]:
t, r_vr = runge_kutta(mov_planetario, np.array([1.0, 0.0, 0.1, 0.5]), [0,2*np.pi], 1.0e-4)

fig = plt.figure()
ax = fig.add_subplot(111)

ax.plot(r_vr[:,0], r_vr[:,1])
ax.set_aspect('equal');

## Runge Kutta de 4o Órden

Esta dado por:
:<math>\begin{align}
y_{n+1} &= y_n + \tfrac{h}{6}\left(k_1 + 2k_2 + 2k_3 + k_4 \right),\\
t_{n+1} &= t_n + h \\
\end{align}</math>

:<math>
\begin{align}
 k_1 &= f(t_n, y_n), \\
 k_2 &= f(t_n + \frac{h}{2}, y_n + \frac{h}{2} k_1), \\
 k_3 &= f(t_n + \frac{h}{2}, y_n + \frac{h}{2} k_2), \\
 k_4 &= f(t_n + h, y_n + hk_3).
\end{align}
</math>

In [ ]:
def rk4(ec,p_ini,x,h=0.1):
    tiempos = np.arange(x[0],x[1]+h,h)
    sol = np.zeros((len(tiempos),len(p_ini)))
    sol[0,:] = p_ini
    for i in xrange(len(tiempos)-1):
        k1 = ec(sol[i,:],tiempos[i])
        k2 = ec(sol[i,:]+0.5*h*k1,tiempos[i]+0.5*h)
        k3 = ec(sol[i,:]+0.5*h*k2,tiempos[i]+0.5*h)
        k4 = ec(sol[i,:]+h*k3,tiempos[i]+h)
        sol[i+1,:]=sol[i,:]+(1/6.0)*h*(k1+2*k2+2*k3+k4)
    return tiempos,sol

t, r_vr = rk4(mov_planetario, np.array([1.0, 0.0, 0.1, 0.5]), [0,2*np.pi], 0.1)

fig = plt.figure()
ax = fig.add_subplot(111)

ax.plot(r_vr[:,0], r_vr[:,1])
ax.set_aspect('equal');

In [ ]:
def osc(y,t,w02=1.):
    return np.array([y[1],-w02*np.sin(y[0])+0.1*np.sin(t)]) 

In [ ]:
t,sol=rk4(osc,[0.5,0.5],[0,10])
plt.plot(sol[:,0],sol[:,1])
zoom = 2
plt.xlim([-zoom,zoom]),plt.ylim([-zoom,zoom])
plt.axes().set_aspect('equal'),plt.grid(True)
plt.show()

In [ ]:
import pylab as pl

In [ ]:
for i in range(10): #el número entre paréntesis es la cantidad de trayectorias
    y0,u0 = np.pi*(2*pl.rand()-1),0 #,4*(2*rand()-1)
    t,sol=rk4(osc,[y0,u0],[0,100])
    plt.plot(sol[:,0],sol[:,1])
zoom = np.pi
plt.xlim([-zoom,zoom]),plt.ylim([-zoom,zoom])
plt.axes().set_aspect('equal'),plt.grid(True)
plt.show()

In [ ]:
y0,u0 = 1.2,0 
t,sol=rk4(osc,[y0,u0],[0,10000.])
plt.plot(sol[:,0],sol[:,1])
plt.plot(sol[:,0],sol[:,1])

y0,u0 = 1.5,0
t,sol=rk4(osc,[y0,u0],[0,10000.])
plt.plot(sol[:,0],sol[:,1])

zoom = np.pi
plt.xlim([-zoom,zoom]),plt.ylim([-zoom,zoom])
plt.axes().set_aspect('equal'),plt.grid(True)
plt.show()